### Курсовая работа Малютина Алексея

Сервисная компания, обслуживает оборудование в удаленных пунктах присутствия компании. В рамках услуги осуществляются выезды специалистов сервисной компании, доставка оборудования и его замена, которая производится со складов и точек присутствия поставщика. По договору стоимость услуги включает транспортные расходы и поставщик предоставил таблицу расстояний от своих складов до пунктов присутствия. В таблице три поля: адрес обслуживаемого подразделения, адрес склада поставщика и расстояние между двумя точками. Необходимо проверить актуальность и корректность полученных данных. Для этого необходимо независимо определить расстояние и сравнить с представленным. 

Т.к. задача разовая, однако объем данных довольно большой, то использование API сервисов карт нецелесообразно. Для решения задачи планируется осуществлить парсинг расстояний по дорогам по картам одного из сервисов, а затем сохранить полученные расстояния в датасет, чтобы сравнить с представленными. Для парсинга будем использовать Selenium with Python. 

В в дополнение к парсингу, используя API разработчика карт будем считывать координаты точек. Результаты отобразим на карте, нанеся расстояния на карту субъекта (республики).

In [244]:
from selenium import webdriver
import pandas as pd
import numpy as np
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from selenium.webdriver import Firefox
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.firefox.options import Options
from selenium.common import exceptions
from shapely.geometry import Polygon, Point, LineString
import time
import warnings
import requests
warnings.filterwarnings("ignore")

In [245]:
dt = pd.read_csv('distance_2_1.csv', delimiter=';')
dt.addr3 = dt.addr3.str.replace('Республика','')
dt['Distance1'] = 0.0
dt['long1'] = 0.0
dt['lat1'] = 0.0
dt['long3'] = 0.0
dt['lat3'] = 0.0

In [246]:
dt.head()

,addr1,addr3,distance,Distance1,long1,lat1,long3,lat3
0,"Мордовия с.Старое Шайгово, ул.Ленина, 7","Мордовия, г. Саранск, ул. Советская, 109",62.0,0.0,0.0,0.0,0.0,0.0
1,"Мордовия г.Инсар, ул.Московская, 91Б","Мордовия, г. Саранск, ул. Советская, 109",74.0,0.0,0.0,0.0,0.0,0.0
2,"Мордовия пгт.Кадошкино, ул.Крупской, 1","Мордовия, г. Саранск, ул. Советская, 109",74.0,0.0,0.0,0.0,0.0,0.0
3,"Мордовия с.Лямбирь, ул.Ленина, 40","Мордовия, г. Саранск, ул. Советская, 109",15.0,0.0,0.0,0.0,0.0,0.0
4,"Мордовия с.Ельники, площадь 1 Мая, 35","Мордовия, г. Краснослободск, ул. Интернационал...",32.0,0.0,0.0,0.0,0.0,0.0


In [247]:
# Преобразуем строку в вещ. число
def str_to_num(str):
    if '.' in str and str.replace('.', '').isdigit():
        return float(str)
    elif str.isdigit():
        return float(str)
    else:
        return 0.0

In [248]:
# Получаем координаты по адресам

API_KEY = "f23814c0-bc5a-46ba-8613-fe4276e7b305"

def get_address(adr):

    URL = f"https://geocode-maps.yandex.ru/1.x/?apikey={API_KEY}&geocode={adr}&format=json&sco=latlong&kind=house&results=1&lang=ru_RU"
    result = requests.get(URL).json()
    return result['response']['GeoObjectCollection']['featureMember'][0]['GeoObject']['Point']['pos']

In [249]:
# Заполним адреса на форме и отправим ее. Полученное расстояние сохраним в датасет
def calc_distance(dt_in, delay, tbr=0):
    dt = dt_in.copy()
    t1 = time.time()
    if tbr == 0:
        browser = webdriver.Safari()
    elif tbr == 1:
        browser = webdriver.Chrome(executable_path=r"D:\Alexey\project\chromedriver.exe")
    elif tbr == 2:
        browser=webdriver.Firefox(executable_path=r"D:\Alexey\project\geckodriver.exe")
    browser.get("https://yandex.ru/maps")
    distance = browser.find_element_by_class_name("route-control")
    distance.click()
    time.sleep(2)
    count_record = 0
    count_except = 0
    for i in range(0, len(dt)):
        if dt['Distance1'][i] == 0 or dt['Distance1'][i] == np.nan: 
            try:
                field_from = browser.find_element_by_xpath("//input[@placeholder='Откуда']")
                time.sleep(delay)
                field_from.send_keys(Keys.COMMAND + "a")
                time.sleep(delay)
                field_from.send_keys(Keys.DELETE)
                time.sleep(delay)
                field_from.send_keys(dt['addr3'][i])
                time.sleep(delay)
                field_from.send_keys(Keys.DOWN)
                time.sleep(delay)
                field_from.send_keys(Keys.ENTER)
                time.sleep(delay)
                field_to = browser.find_element_by_xpath("//input[@placeholder='Куда']")
                time.sleep(delay)
                field_to.send_keys(Keys.COMMAND + "a")
                time.sleep(delay)
                field_to.send_keys(Keys.DELETE)
                time.sleep(delay)
                field_to.send_keys(dt['addr1'][i])
                time.sleep(delay)
                field_to.send_keys(Keys.DOWN)
                time.sleep(delay)
                field_to.send_keys(Keys.ENTER)
                time.sleep(delay)
                dist_km = ''
                for element in browser.find_elements_by_xpath("//*[contains(text(), 'км')]"):
                    dist_km += '' + element.text
                # print(dist_km.split('км')[1])
                rasst = dist_km.split('км')[1].replace('\xa0', '')
                rasst = rasst.replace(',','.')
                # print(rasst, end='\t')
                dt['Distance1'][i] = str_to_num(rasst)
                # print(dt['Distance1'][i])
                count_record += 1
                t2 = time.time()
                geo3 = []
                geo3 = get_address(dt['addr3'][i]).split(' ')
                geo1 = []
                geo1 = get_address(dt['addr1'][i]).split(' ')
                dt['long3'][i] = float(geo3[0])
                dt['lat3'][i] = float(geo3[1])
                dt['long1'][i] = float(geo1[0])
                dt['lat1'][i] = float(geo1[1])
                if i % 100 == 0: 
                    t = t2 - t1
                    print(f'{count_record} записей обработано')
                    print(f'{count_except} ошибок возникло')
                    print(f'Время обработки - {t:8.2f} c.')
            except (exceptions.StaleElementReferenceException, exceptions.NoSuchElementException):
                count_except += 1
                pass 
    browser.close()
    t2 = time.time()
    t = t2 - t1
    print(f'Всего {count_record} записей обработано')
    print(f'Всего {count_except} ошибок возникло')
    print(f'Общее время обработки - {t:8.2f} c.')
    print('\n')
    return dt



In [250]:
# Парсим расстояния по дорогам
dt_out =  calc_distance(dt[:10], 0.5, 0)

1 записей обработано
0 ошибок возникло
Время обработки -    14.16 c.
Всего 10 записей обработано
Всего 0 ошибок возникло
Общее время обработки -    94.66 c.




In [251]:
dt_out

,addr1,addr3,distance,Distance1,long1,lat1,long3,lat3
0,"Мордовия с.Старое Шайгово, ул.Ленина, 7","Мордовия, г. Саранск, ул. Советская, 109",62.0,58.0,44.467397,54.303891,45.166807,54.182591
1,"Мордовия г.Инсар, ул.Московская, 91Б","Мордовия, г. Саранск, ул. Советская, 109",74.0,74.0,44.366480,53.869863,45.166807,54.182591
2,"Мордовия пгт.Кадошкино, ул.Крупской, 1","Мордовия, г. Саранск, ул. Советская, 109",74.0,74.0,44.418735,54.026208,45.166807,54.182591
3,"Мордовия с.Лямбирь, ул.Ленина, 40","Мордовия, г. Саранск, ул. Советская, 109",15.0,15.0,45.115523,54.293445,45.166807,54.182591
4,"Мордовия с.Ельники, площадь 1 Мая, 35","Мордовия, г. Краснослободск, ул. Интернационал...",32.0,31.0,43.862768,54.619521,43.792403,54.422900
5,"Мордовия с.Атюрьево, ул.Кирова, 4а","Мордовия, г. Краснослободск, ул. Интернационал...",36.0,36.0,43.333373,54.321219,43.792403,54.422900
6,"Мордовия г.Темников, ул.Гражданская, 2","Мордовия, г. Краснослободск, ул. Интернационал...",62.0,61.0,43.205201,54.629549,43.792403,54.422900
7,"Мордовия р.п.Торбеево, ул.Октябрьская, 18","Мордовия, пгт. Зубова Поляна, ул. Пролетарская, 6",40.0,37.0,43.245419,54.073243,42.844114,54.079630
8,"Мордовия пгт.Комсомольский, Микрорайон-2, 30","Мордовия, пгт. Чамзинка, ул. Победы, 2",5.0,4.6,45.811672,54.433119,45.775982,54.404090
9,"Мордовия с.Дубенки, ул.Центральная, 19","Мордовия, пгт. Чамзинка, ул. Победы, 2",42.0,38.0,46.300553,54.432558,45.775982,54.404090


In [252]:
# Проверяем кол-во записей с разницей расстояний более 100 км (вероятно, ошибка расчета расстояния или парсинга)
dt_out[abs(dt_out['Distance1'].astype(float) - dt_out['distance'].astype(float)) > 100].count()

addr1        0
addr3        0
distance     0
Distance1    0
long1        0
lat1         0
long3        0
lat3         0
dtype: int64

In [253]:
# Устанавливаем для записей с разницей заявленного и рассчитанного растояние более 100 км значение 0
# После этого, в случае необходимости, выполняем ячейук парсинга расстояния, увеличив время между нажатиями клавиш
dt_out[abs(dt_out['Distance1'].astype(float) - dt_out['distance'].astype(float)) > 100]['Distance1'] = 0.0

In [254]:
# Повторно парсим расстояния по дорогам, увеличив время между нажатиями клавиш
dt_out =  calc_distance(dt_out, 1, 0)

Всего 0 записей обработано
Всего 0 ошибок возникло
Общее время обработки -     6.40 c.




In [255]:
dt_out

,addr1,addr3,distance,Distance1,long1,lat1,long3,lat3
0,"Мордовия с.Старое Шайгово, ул.Ленина, 7","Мордовия, г. Саранск, ул. Советская, 109",62.0,58.0,44.467397,54.303891,45.166807,54.182591
1,"Мордовия г.Инсар, ул.Московская, 91Б","Мордовия, г. Саранск, ул. Советская, 109",74.0,74.0,44.366480,53.869863,45.166807,54.182591
2,"Мордовия пгт.Кадошкино, ул.Крупской, 1","Мордовия, г. Саранск, ул. Советская, 109",74.0,74.0,44.418735,54.026208,45.166807,54.182591
3,"Мордовия с.Лямбирь, ул.Ленина, 40","Мордовия, г. Саранск, ул. Советская, 109",15.0,15.0,45.115523,54.293445,45.166807,54.182591
4,"Мордовия с.Ельники, площадь 1 Мая, 35","Мордовия, г. Краснослободск, ул. Интернационал...",32.0,31.0,43.862768,54.619521,43.792403,54.422900
5,"Мордовия с.Атюрьево, ул.Кирова, 4а","Мордовия, г. Краснослободск, ул. Интернационал...",36.0,36.0,43.333373,54.321219,43.792403,54.422900
6,"Мордовия г.Темников, ул.Гражданская, 2","Мордовия, г. Краснослободск, ул. Интернационал...",62.0,61.0,43.205201,54.629549,43.792403,54.422900
7,"Мордовия р.п.Торбеево, ул.Октябрьская, 18","Мордовия, пгт. Зубова Поляна, ул. Пролетарская, 6",40.0,37.0,43.245419,54.073243,42.844114,54.079630
8,"Мордовия пгт.Комсомольский, Микрорайон-2, 30","Мордовия, пгт. Чамзинка, ул. Победы, 2",5.0,4.6,45.811672,54.433119,45.775982,54.404090
9,"Мордовия с.Дубенки, ул.Центральная, 19","Мордовия, пгт. Чамзинка, ул. Победы, 2",42.0,38.0,46.300553,54.432558,45.775982,54.404090


In [256]:
# Число записей, для которых не удалось определить расстояние
dt_out[dt_out['Distance1'] == 0].groupby(['Distance1']).count()

,addr1,addr3,distance,long1,lat1,long3,lat3
Distance1,,,,,,,


In [257]:
# Находим разницу в исходном и полученном расстоянии. 
# Если разность > 0, то у поставшика услуг заявлено большее расстояние, чем полученное в ходе парсинга
dt_out['Diff'] = dt_out['distance'] - dt_out['Distance1']

In [258]:
# Создаем дополненный расстояниями файл
path = "distance_3_1.csv"
dt_out.to_csv(path)

In [259]:
# Отображаем схему транспортных маршрутов на карте
import json
import folium
import pandas as pd
from folium.features import DivIcon

with open('./get_geojson.txt') as json_file:
    mordovia_polygon = json.load(json_file)
    
with open('./get_geojson1.txt') as json_file:
    chuvashia_polygon = json.load(json_file)
    
with open('./get_geojson2.txt') as json_file:
    mari_polygon = json.load(json_file)

m = folium.Map(location=[56.07, 47.15], # Чебоксары
               zoom_start=6)

m.choropleth(mordovia_polygon, fill_color='yellow')
m.choropleth(chuvashia_polygon, fill_color='brown')
m.choropleth(mari_polygon, fill_color='blue')
for i, x in dt_out.iterrows():
    folium.PolyLine(locations = [(x[5], x[4]), (x[7], x[6])], line_opacity = 0.5).add_to(m)

In [260]:
m